### Q1

In [2]:
from queue import PriorityQueue

class Node:
    def __init__(self, position, parent=None, visited_goals=None):
        self.position = position
        self.parent = parent
        self.visited_goals = visited_goals if visited_goals else set()
        self.g = 0  #cost from start node to curr node
        self.h = 0  #est. heuristic cost to goal
        self.f = 0  #total est. cost

    def __lt__(self, other):
        return self.f < other.f  #priority queue comparison

def heuristic(current_pos, goals): #compute heuristic as min manhattan dist to any unvisited goal
    return min(abs(current_pos[0] - g[0]) + abs(current_pos[1] - g[1]) for g in goals if g not in current_pos)

def best_first_search_multi_goal(maze, start, goals): #best-first search to find shortest path covering all goals
    rows, cols = len(maze), len(maze[0])
    initial_node = Node(start, visited_goals=set())
    frontier = PriorityQueue()
    frontier.put((0, initial_node))
    
    visited_states = set()

    while not frontier.empty():
        _, current_node = frontier.get()
        current_pos = current_node.position
        visited_goals = current_node.visited_goals.copy()

        #marking goal as visited, if reached
        if current_pos in goals:
            visited_goals.add(current_pos)

        #if all goals are visited, then reconstruct path
        if visited_goals == set(goals):
            path = []
            while current_node:
                path.append(current_node.position)
                current_node = current_node.parent
            return path[::-1]  #reverse path to start from initial pos

        #prevent revisiting the same state(pos + visited goals)
        state = (current_pos, frozenset(visited_goals))
        if state in visited_states:
            continue
        visited_states.add(state)

        #expand adjacent nodes
        for dx, dy in [(1, 0), (-1, 0), (0, 1), (0, -1)]:
            new_pos = (current_pos[0] + dx, current_pos[1] + dy)
            if 0 <= new_pos[0] < rows and 0 <= new_pos[1] < cols and maze[new_pos[0]][new_pos[1]] == 0:
                new_node = Node(new_pos, current_node, visited_goals)
                new_node.g = current_node.g + 1
                new_node.h = heuristic(new_pos, goals)
                new_node.f = new_node.h 
                frontier.put((new_node.f, new_node))

    return None

maze = [
    [0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 1, 0, 1],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 1, 0]
]

start = (0, 0)
goals = [(4, 4), (2, 3), (1, 4)] 

print("best-first search for multiple goals:")
path = best_first_search_multi_goal(maze, start, goals)

if path:
    print("path found:", path)
else:
    print("no path found")


best-first search for multiple goals:
path found: [(0, 0), (1, 0), (1, 1), (1, 2), (1, 3), (1, 4), (1, 3), (2, 3), (3, 3), (3, 4), (4, 4)]


### Q2

In [3]:
from queue import PriorityQueue
import random
import time

class Graph:
    def __init__(self):
        self.graph = {
            'A': {'B': 2, 'C': 4},
            'B': {'D': 7, 'E': 3},
            'C': {'F': 1, 'G': 5},
            'D': {'H': 2},
            'E': {},
            'F': {'I': 6},
            'G': {},
            'H': {},
            'I': {}
        }
        self.heuristic = {  
            'A': 10, 'B': 8, 'C': 6, 'D': 4, 'E': 5,
            'F': 3, 'G': 6, 'H': 2, 'I': 0
        }

    def get_neighbors(self, node):
        return self.graph[node]

    def update_edge_cost(self):
        node = random.choice(list(self.graph.keys()))
        if self.graph[node]:  #ensure node has neighbors
            neighbor = random.choice(list(self.graph[node].keys()))
            new_cost = random.randint(1, 10)  #random new cost
            self.graph[node][neighbor] = new_cost
            print(f"Edge cost updated: {node} -> {neighbor} now costs {new_cost}")

def a_star_dynamic(graph_obj, start, goal):
    graph = graph_obj.graph
    heuristic = graph_obj.heuristic

    frontier = PriorityQueue()
    frontier.put((heuristic[start], 0, start))  #(f(n), g(n), node)
    
    came_from = {start: None}
    g_costs = {start: 0}

    while not frontier.empty():
        _, g_current, current = frontier.get()

        print(f"Expanding: {current}")

        if current == goal:
            path = []
            while current:
                path.append(current)
                current = came_from[current]
            path.reverse()
            print(f"Optimal Path Found: {path}")
            return path

        #simulate dynamic edge cost change (every few expansions)
        if random.random() < 0.3:  # 30% chance to update costs
            graph_obj.update_edge_cost()

        #explore neighbors
        for neighbor, cost in graph[current].items():
            new_g_cost = g_current + cost  #updated g(n)
            f_cost = new_g_cost + heuristic[neighbor]  #f(n)=g(n)+h(n)

            if neighbor not in g_costs or new_g_cost < g_costs[neighbor]:
                g_costs[neighbor] = new_g_cost
                came_from[neighbor] = current
                frontier.put((f_cost, new_g_cost, neighbor))

    print("goal not reachable")
    return None

graph = Graph()

print("Running A* Search with dynamic edge costs:")
a_star_dynamic(graph, 'A', 'I')


Running A* Search with dynamic edge costs:
Expanding: A
Expanding: B
Edge cost updated: F -> I now costs 5
Expanding: C
Expanding: F
Expanding: E
Edge cost updated: A -> B now costs 7
Expanding: I
Optimal Path Found: ['A', 'C', 'F', 'I']


['A', 'C', 'F', 'I']

### Q3

In [5]:
from queue import PriorityQueue

class DeliveryGraph:
    def __init__(self):
        self.graph = {
            'Warehouse': {'A': 4, 'B': 2, 'C': 5},
            'A': {'D': 3, 'E': 7},
            'B': {'F': 4},
            'C': {'G': 6},
            'D': {'H': 2},
            'E': {},
            'F': {'I': 5},
            'G': {},
            'H': {},
            'I': {}
        }

        self.time_windows = {
            'A': 8, 'B': 5, 'C': 10, 'D': 3, 'E': 12,
            'F': 6, 'G': 9, 'H': 2, 'I': 4
        }

        self.heuristic = {
            'Warehouse': 10, 'A': 6, 'B': 4, 'C': 7, 'D': 3, 
            'E': 8, 'F': 5, 'G': 6, 'H': 2, 'I': 0
        }

    def get_neighbors(self, node):
        return self.graph[node]

def greedy_best_first_search(graph_obj, start, goal):
    graph = graph_obj.graph
    heuristic = graph_obj.heuristic
    time_windows = graph_obj.time_windows

    frontier = PriorityQueue()
    frontier.put((heuristic[start], start))  #(h(n), node)
    
    visited = set()
    came_from = {start: None}

    while not frontier.empty():
        _, current = frontier.get()

        print(f"delivering to: {current} (deadline: {time_windows.get(current, 'N/A')})")

        if current == goal:
            path = []
            while current:
                path.append(current)
                current = came_from[current]
            path.reverse()
            print(f"optimal delivery route: {path}")
            return path

        visited.add(current)

        #expand neighbors & prioritize those w/ stricter time constraints
        deliveries = []
        for neighbor in graph[current]:
            if neighbor not in visited:
                deliveries.append((time_windows[neighbor], heuristic[neighbor], neighbor))

        #sort by (earliest deadline, shortest est. dist)
        deliveries.sort()

        for _, _, neighbor in deliveries:
            came_from[neighbor] = current
            frontier.put((heuristic[neighbor], neighbor))

    print("no delivery route found.")
    return None

graph = DeliveryGraph()

print("running greedy best-first search for delivery route optimization:")
greedy_best_first_search(graph, 'Warehouse', 'I')


running greedy best-first search for delivery route optimization:
delivering to: Warehouse (deadline: N/A)
delivering to: B (deadline: 5)
delivering to: F (deadline: 6)
delivering to: I (deadline: 4)
optimal delivery route: ['Warehouse', 'B', 'F', 'I']


['Warehouse', 'B', 'F', 'I']